> **How to run this notebook (command-line)?**
1. Install the `ReinventCommunity` environment:
`conda env create -f environment.yml`
2. Activate the environment:
`conda activate ReinventCommunity`
3. Execute `jupyter`:
`jupyter notebook`
4. Copy the link to a browser


# `REINVENT 3.2`: Model building demo
For many applications of `REINVENT`, we already have some prior knowledge on a project that we would like to incorporate. One example of how this can be achieved are *predictive models* trained on a collection of compounds, for example `QSAR` models that relate the structure of a compound to an activity / potency endpoint. In this demo, we will explain how to build `scikit-learn` models that are compatible with `REINVENT`. Note, that our model will have the ending `.pkl`, as we need to save the model's parameters in a serialized fashion ("[pickled](https://docs.python.org/3/library/pickle.html)").

Our input dataset will be `DRD2` (see also the "complete use-case notebook"), for which we have compiled two `CSV` files with the compounds' `SMILES` in one column and a read-out value in another (see below). We will train a `Random Forest` regressor here, but you can choose any backend algorithm that provides a `scikit-learn` interface. As of now, `REINVENT` supports 4 different kinds of molecular fingerprints and we will need to calculate them before we can train our model.

The following imports are required to execute the code. Please update the output directory path if you want to store the temporary files and the final model in another place.

In [ ]:
import os
import pandas as pd
import numpy as np
import random
import pickle
import sklearn.ensemble
from sklearn.metrics import roc_auc_score, mean_squared_error

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, MACCSkeys, PandasTools
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem.Draw import IPythonConsole
from IPython.core.display import display, HTML

from shutil import copyfile

import matplotlib.pyplot as plt
import seaborn as sns

# set plotting parameters
large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (16, 10),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")
%matplotlib inline

In [ ]:
print("The current working directory is", os.getcwd())

# --------- change these path variables as required
output_dir = os.path.expanduser("./REINVENT_model_building_demo")

# --------- do not change
# get the notebook's root path
try: ipynb_path
except NameError: ipynb_path = os.getcwd()

# if required, generate a folder to store the results
try:
    os.mkdir(output_dir)
except FileExistsError:
    pass

# copy data sets to temporary folder for inspection
copyfile(os.path.join(ipynb_path, "data/drd2.train.csv"),
         os.path.join(output_dir, "drd2.train.csv"))
copyfile(os.path.join(ipynb_path, "data/drd2.test.csv"),
         os.path.join(output_dir, "drd2.test.csv"));

The current working directory is /home/springnuance/reinvent-hitl/Reinvent-Community-Binh/notebooks


## Data
Before proceeding, let us inspect the datasets. We have split them into a training and test sets, each of which has `canonical` (the `SMILES`) and `activity` (either `1` or `0`, corresponding to `active` and `inactive` respectively) columns. This means, we have a (binary) classification problem.

In [ ]:
train = pd.read_csv(os.path.join(output_dir, "drd2.train.csv"))
test = pd.read_csv(os.path.join(output_dir, "drd2.test.csv"))


print("# obs in train: ", train.shape[0])
print("# obs in test: ", test.shape[0])
train.head()

# obs in train:  275768
# obs in test:  68944


,canonical,activity
0,COc1ccc(NC(=O)CC2C(=O)N(c3ccc(Cl)cc3)C(=S)N2CC...,0
1,O=C(CSc1oc(-c2ccccc2)nc1S(=O)(=O)c1ccc(Br)cc1)...,0
2,Cn1c(=S)n(CCC(=O)O)c2ccccc21,0
3,Clc1ccc(NN=Cc2ccncc2)cc1Cl,0
4,O=C(C=NO)NCCCNCc1ccccc1,0


In [ ]:
print("Unique values in activity")
print("We can see that activity = 1 is extremely rare compared to activity = 0")
train["activity"].value_counts()



Unique values in activity
We can see that activity = 1 is extremely rare compared to activity = 0


0    272320
1      3448
Name: activity, dtype: int64

## Descriptors
As mentioned above, we need to calculate descriptors (fingerprints) which are understood by `REINVENT`. Below is some code to facilitate this for one example configuration (`ECFP6` with counts), but feel free to adapt it if you feel your model could benefit from it.

In [ ]:
def smiles_to_mols(query_smiles):

    mols = [Chem.MolFromSmiles(smile) for smile in query_smiles]
    valid = [0 if mol is None else 1 for mol in mols]
    valid_idxs = [idx for idx, boolean in enumerate(valid) if boolean == 1]
    valid_mols = [mols[idx] for idx in valid_idxs]
    return valid_mols, valid_idxs

def smiles_to_mols_batches(query_smiles, batch_size=10000):
    batched_mols = []
    batched_idxs = []
    # print(len(query_smiles))
    for i in range(0, len(query_smiles), batch_size):
        if i % 100 == 0:
            print(f"Processing batch {i} to {i+batch_size}")
        batch = query_smiles[i:i+batch_size]
        mols, idxs = smiles_to_mols(batch)
        batched_mols.extend(mols)
        # Adjust index according to the batch
        adjusted_idxs = [i + idx for idx in idxs]
        batched_idxs.extend(adjusted_idxs)
    return batched_mols, batched_idxs

def getMorganFingerprints(mols, radius, useFeatures, useCounts):
    fps = [AllChem.GetMorganFingerprint(mol, radius, useCounts=useCounts, useFeatures=useFeatures) for mol in mols]
    return fps

class Descriptors:

    def __init__(self, data):
        self._data = data

    def ECFP(self, radius, nBits):
        fingerprints = []
        mols, idx = smiles_to_mols(self._data)
        
        fp_bits = [AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits) for mol in mols]
        
        for fp in fp_bits:
            fp_np = np.zeros((1, nBits), dtype=np.int32)
            DataStructs.ConvertToNumpyArray(fp, fp_np)
            fingerprints.append(fp_np)
        return fingerprints, idx

    def ECFP_counts(self, radius, useFeatures, useCounts=True):
        
        #mols, valid_idx = smiles_to_mols(self._data)
        mols, valid_idx = smiles_to_mols_batches(self._data)
        print("Finish converting smiles to mols")
        fps = [AllChem.GetMorganFingerprint(mol, radius, useCounts=useCounts, useFeatures=useFeatures) for mol in mols]
        print("Finish getting Morgan fingerprints")
        size = 2048
        nfp = np.zeros((len(fps), size), np.int32)
        print("The shape of nfp is", nfp.shape)
        for i, fp in enumerate(fps):
            for idx, v in fp.GetNonzeroElements().items():
                nidx = idx % size
                nfp[i, nidx] += int(v)
        return nfp, valid_idx

    def Avalon(self, nBits):
        mols, valid_idx = smiles_to_mols(self._data)
        fingerprints = []
        fps = [pyAvalonTools.GetAvalonFP(mol, nBits=nBits) for mol in mols]
        for fp in fps:
            fp_np = np.zeros((1, nBits), dtype=np.int32)
            DataStructs.ConvertToNumpyArray(fp, fp_np)
            fingerprints.append(fp_np)
        return fingerprints, valid_idx

    def MACCS_keys(self):
        mols, valid_idx = smiles_to_mols(self._data)
        fingerprints = []
        fps = [MACCSkeys.GenMACCSKeys(mol) for mol in mols]
        for fp in fps:
            fp_np = np.zeros((1, ), dtype=np.int32)
            DataStructs.ConvertToNumpyArray(fp, fp_np)
            fingerprints.append(fp_np)
        return fingerprints, valid_idx
    
def get_ECFP6_counts(inp):
    if not isinstance(inp, list):
        inp = list(inp)
    desc = Descriptors(inp)
    fps, _ = desc.ECFP_counts(radius=3, useFeatures=True, useCounts=True)
    return fps

### Getting valid mols and valid idxs

In [ ]:
train_valid_mols, train_valid_idxs = smiles_to_mols_batches(train["canonical"], batch_size=10000)
np.save(os.path.join(output_dir, "mols_and_idxs/train_valid_mols.npy"), train_valid_mols)
np.save(os.path.join(output_dir, "mols_and_idxs/train_valid_idxs.npy"), train_valid_idxs)

In [ ]:
test_valid_mols, test_valid_idxs = smiles_to_mols_batches(test["canonical"], batch_size=10000)
np.save(os.path.join(output_dir, "mols_and_idxs/test_valid_mols.npy"), test_valid_mols)
np.save(os.path.join(output_dir, "mols_and_idxs/test_valid_idxs.npy"), test_valid_idxs)

### Getting the Morgan Fingerprints

In [ ]:
train_valid_mols = np.load(os.path.join(output_dir, "mols_and_idxs/train_valid_mols.npy"), allow_pickle=True).tolist()
train_fps = getMorganFingerprints(train_valid_mols, radius=3, useFeatures=True, useCounts=True)
np.save(os.path.join(output_dir, "Morgan_fingerprints/train_fps.npy"), train_fps)

In [ ]:
test_valid_mols = np.load(os.path.join(output_dir, "mols_and_idxs/test_valid_mols.npy"), allow_pickle=True).tolist()
test_fps = getMorganFingerprints(test_valid_mols, radius=3, useFeatures=True, useCounts=True)
np.save(os.path.join(output_dir, "Morgan_fingerprints/test_fps.npy"), test_fps)

### Getting Nonzero Morgan Fingerprints

In [ ]:
batch_size = 10000
size = 2048

train_fps = np.load(os.path.join(output_dir, "Morgan_fingerprints/train_fps.npy"), allow_pickle=True).tolist()

num_batches = len(train_fps) // batch_size + (1 if len(train_fps) % batch_size > 0 else 0)

for batch_num in range(num_batches):
    print(f"Processing batch {batch_num + 1}/{num_batches}")
    
    # Calculate start and end index for current batch
    start_idx = batch_num * batch_size
    end_idx = min((batch_num + 1) * batch_size, len(train_fps))
    
    # Initialize the nfp array for this batch
    nfp_batch = np.zeros((end_idx - start_idx, size), np.int16)
    
    for i in range(start_idx, end_idx):
        # Update the progress every 10000 fingerprints within the batch if needed
        if (i - start_idx) % 10000 == 0:
            print(f"  Processing fingerprint {i - start_idx} in batch {batch_num + 1}")
        
        fp = train_fps[i]
        for idx, v in fp.GetNonzeroElements().items():
            nidx = idx % size
            nfp_batch[i - start_idx, nidx] += int(v)
    
    # Save the current batch
    batch_filename = os.path.join(output_dir, f"nonzero_Morgan_fingerprints/train_nfp_{batch_num + 1}.npy")
    np.save(batch_filename, nfp_batch)
    print(f"Batch {batch_num + 1} saved as {batch_filename}")

Processing batch 1/28
  Processing fingerprint 0 in batch 1
Batch 1 saved as ./REINVENT_model_building_demo/nonzero_Morgan_fingerprints/train_nfp_1.npy
Processing batch 2/28
  Processing fingerprint 0 in batch 2
Batch 2 saved as ./REINVENT_model_building_demo/nonzero_Morgan_fingerprints/train_nfp_2.npy
Processing batch 3/28
  Processing fingerprint 0 in batch 3
Batch 3 saved as ./REINVENT_model_building_demo/nonzero_Morgan_fingerprints/train_nfp_3.npy
Processing batch 4/28
  Processing fingerprint 0 in batch 4
Batch 4 saved as ./REINVENT_model_building_demo/nonzero_Morgan_fingerprints/train_nfp_4.npy
Processing batch 5/28
  Processing fingerprint 0 in batch 5
Batch 5 saved as ./REINVENT_model_building_demo/nonzero_Morgan_fingerprints/train_nfp_5.npy
Processing batch 6/28
  Processing fingerprint 0 in batch 6
Batch 6 saved as ./REINVENT_model_building_demo/nonzero_Morgan_fingerprints/train_nfp_6.npy
Processing batch 7/28
  Processing fingerprint 0 in batch 7
Batch 7 saved as ./REINVENT_

In [ ]:
batch_size = 10000
size = 2048

test_fps = np.load(os.path.join(output_dir, "Morgan_fingerprints/test_fps.npy"), allow_pickle=True).tolist()

num_batches = len(test_fps) // batch_size + (1 if len(test_fps) % batch_size > 0 else 0)

for batch_num in range(num_batches):
    print(f"Processing batch {batch_num + 1}/{num_batches}")
    
    # Calculate start and end index for current batch
    start_idx = batch_num * batch_size
    end_idx = min((batch_num + 1) * batch_size, len(test_fps))
    
    # Initialize the nfp array for this batch
    nfp_batch = np.zeros((end_idx - start_idx, size), np.int16)
    
    for i in range(start_idx, end_idx):
        # Update the progress every 10000 fingerprints within the batch if needed
        if (i - start_idx) % 10000 == 0:
            print(f"  Processing fingerprint {i - start_idx} in batch {batch_num + 1}")
        
        fp = test_fps[i]
        for idx, v in fp.GetNonzeroElements().items():
            nidx = idx % size
            nfp_batch[i - start_idx, nidx] += int(v)
    
    # Save the current batch
    batch_filename = os.path.join(output_dir, f"nonzero_Morgan_fingerprints/test_nfp_{batch_num + 1}.npy")
    np.save(batch_filename, nfp_batch)
    print(f"Batch {batch_num + 1} saved as {batch_filename}")


Processing batch 1/7
  Processing fingerprint 0 in batch 1
Batch 1 saved as ./REINVENT_model_building_demo/nonzero_Morgan_fingerprints/test_nfp_1.npy
Processing batch 2/7
  Processing fingerprint 0 in batch 2
Batch 2 saved as ./REINVENT_model_building_demo/nonzero_Morgan_fingerprints/test_nfp_2.npy
Processing batch 3/7
  Processing fingerprint 0 in batch 3
Batch 3 saved as ./REINVENT_model_building_demo/nonzero_Morgan_fingerprints/test_nfp_3.npy
Processing batch 4/7
  Processing fingerprint 0 in batch 4
Batch 4 saved as ./REINVENT_model_building_demo/nonzero_Morgan_fingerprints/test_nfp_4.npy
Processing batch 5/7
  Processing fingerprint 0 in batch 5
Batch 5 saved as ./REINVENT_model_building_demo/nonzero_Morgan_fingerprints/test_nfp_5.npy
Processing batch 6/7
  Processing fingerprint 0 in batch 6
Batch 6 saved as ./REINVENT_model_building_demo/nonzero_Morgan_fingerprints/test_nfp_6.npy
Processing batch 7/7
  Processing fingerprint 0 in batch 7
Batch 7 saved as ./REINVENT_model_buildin

## Model training
To train your model (i.e. fit the model's parameters to the training data), execute the following cell. Note, that `REINVENT` will access the `proba` property of the model to get a probability rather than a predicted label. If you want to optimize the hyper-parameters of you model, we suggest you use a cross-validation approach (we aim to publish our in-house method based on [optuna](https://optuna.org) soon).

In [ ]:
print(np.array(train_fps).shape)
print(train["activity"].shape)

In [ ]:
train_nfp = []

batch_size = 10000

train_fps = np.load(os.path.join(output_dir, "Morgan_fingerprints/train_fps.npy"), allow_pickle=True).tolist()

num_batches = len(train_fps) // batch_size + (1 if len(train_fps) % batch_size > 0 else 0)

for batch_num in range(num_batches):
    print(f"Adding batch {batch_num + 1}/{num_batches}")
    train_nfp_batch_i = np.load(os.path.join(output_dir, f"nonzero_Morgan_fingerprints/train_nfp_{batch_num + 1}.npy"))
    train_nfp.append(train_nfp_batch_i)

train_nfp = np.concatenate(train_nfp, axis=0)
print(train_nfp.shape)
    

Adding batch 1/28
Adding batch 2/28
Adding batch 3/28
Adding batch 4/28
Adding batch 5/28
Adding batch 6/28
Adding batch 7/28
Adding batch 8/28
Adding batch 9/28
Adding batch 10/28
Adding batch 11/28
Adding batch 12/28
Adding batch 13/28
Adding batch 14/28
Adding batch 15/28
Adding batch 16/28
Adding batch 17/28
Adding batch 18/28
Adding batch 19/28
Adding batch 20/28
Adding batch 21/28
Adding batch 22/28
Adding batch 23/28
Adding batch 24/28
Adding batch 25/28
Adding batch 26/28
Adding batch 27/28
Adding batch 28/28
(275768, 2048)


In [ ]:
test_nfp = []

batch_size = 10000

test_fps = np.load(os.path.join(output_dir, "Morgan_fingerprints/test_fps.npy"), allow_pickle=True).tolist()

num_batches = len(test_fps) // batch_size + (1 if len(test_fps) % batch_size > 0 else 0)

for batch_num in range(num_batches):
    print(f"Adding batch {batch_num + 1}/{num_batches}")
    test_nfp_batch_i = np.load(os.path.join(output_dir, f"nonzero_Morgan_fingerprints/test_nfp_{batch_num + 1}.npy"))
    test_nfp.append(test_nfp_batch_i)

test_nfp = np.concatenate(test_nfp, axis=0)
print(test_nfp.shape)

Adding batch 1/7
Adding batch 2/7
Adding batch 3/7
Adding batch 4/7
Adding batch 5/7
Adding batch 6/7
Adding batch 7/7
(68944, 2048)


In [ ]:
# initialize a "Random Forest Classifier" (choice of hyper-parameters somewhat arbitrary)
RFclassifier = sklearn.ensemble.RandomForestClassifier(max_depth=20,
                                                       max_features="auto",
                                                       n_estimators=100,
                                                       class_weight="balanced")

# fit to training data
RFclassifier.fit(train_nfp, train["activity"])
y_pred = RFclassifier.predict(X=train_nfp)
train_score = roc_auc_score(y_true=train["activity"], y_score=y_pred)
print(train_score)

In [ ]:
### Saving the classifier
with open(os.path.join(output_dir, "trained_classifiers/DRD2_only_train_model.pkl"), "wb") as f:
    pickle.dump(RFclassifier, f)

## Evaluate performance on test set
To get a better idea on how well (or meager) our model performs, we can evaluate it on the test (hold-out) set. These observations have not been used for fitting the parameters and are thus a good proxy for how the model would fare for new compounds generated, e.g. by `REINVENT`. Note, that we do not strive to give a comprehensive tutorial how to build good, well generalizing models in this demonstration and there are quite a few important considerations to contemplate before actually using a model in production.

In [ ]:
RFclassifier = pickle.load(open(os.path.join(output_dir, "trained_classifiers/DRD2_only_train_model.pkl"), "rb"))

y_pred_test = RFclassifier.predict(X=test_nfp)
test_score = roc_auc_score(y_true=test["activity"], y_score=y_pred_test)
print(test_score)

0.9644378348788165


## Write out final model
Finally, we will write-out the model. Usually, we will build it using all the observations available. While this maximizes the amount of knowledge available to the model, it is noteworthy that this comes at the trade-off of not being able to independently assess the model's performance anymore.

In [ ]:
RFclassifier_final = sklearn.ensemble.RandomForestClassifier(max_depth=20,
                                                             max_features="auto",
                                                             n_estimators=100,
                                                             class_weight="balanced")

# fit to all data points
complete_nfp = np.concatenate((train_nfp, test_nfp), axis=0)
complete_y = pd.concat((train["activity"], test["activity"]))
RFclassifier_final.fit(complete_nfp, complete_y)

In [ ]:
# save the model
with open(os.path.join(output_dir, "trained_classifiers/DRD2_final_model.pkl"), "wb") as f:
    pickle.dump(RFclassifier_final, f)

In [ ]:
RFclassifier_final = pickle.load(open(os.path.join(output_dir, "trained_classifiers/DRD2_final_model.pkl"), "rb"))

complete_nfp = np.concatenate((train_nfp, test_nfp), axis=0)
complete_y = pd.concat((train["activity"], test["activity"]))
y_pred_final = RFclassifier_final.predict(X=complete_nfp)
train_score = roc_auc_score(y_true=complete_y, y_score=y_pred_final)
print(train_score)

ValueError: node array from the pickle has an incompatible dtype:
- expected: [('left_child', '<i8'), ('right_child', '<i8'), ('feature', '<i8'), ('threshold', '<f8'), ('impurity', '<f8'), ('n_node_samples', '<i8'), ('weighted_n_node_samples', '<f8')]
- got     : {'names':['left_child','right_child','feature','threshold','impurity','n_node_samples','weighted_n_node_samples','missing_go_to_left'], 'formats':['<i8','<i8','<i8','<f8','<f8','<i8','<f8','u1'], 'offsets':[0,8,16,24,32,40,48,56], 'itemsize':64}

## Integration into `REINVENT`
In order to use your new model as a component in the scoring function of `REINVENT`, you need to include a block with the appropriate parameter settings (see below). Note, that the descriptor definition needs to match.

```
{
  "component_type": "predictive_property",
  "name": "DRD2_pred_activity",
  "weight": 1,
  "specific_parameters": {
    "model_path": "/path/to/model/folder/DRD2_final_model.pkl",
    "scikit": "classification",
    "transformation: {
        "transformation_type": "no_transformation"
    },
    "descriptor_type": "ecfp_counts",
    "size": 2048,
    "radius": 3,
    "use_counts": True,
    "use_features": True
  }
}
```